In [1]:
from pyarrow.parquet import ParquetFile
import pandas as pd
import numpy as np
from tqdm import tqdm


In [5]:
multiome_var_meta = pd.read_csv("../../data/multiome_var_meta.csv")
gene_names = multiome_var_meta[multiome_var_meta["feature_type"] == "Gene Expression"]["location"].to_list()

multiome_df = pd.DataFrame()
batch_num = 10

def get_chr(all_str):
    index = all_str.find(":") 
    return all_str[:index] 

def get_start(all_str):
    return int(all_str[all_str.find(":")+1:all_str.find("-")])

def get_end(all_str):
    return int(all_str[all_str.find("-")+1:])

multiome_file = ParquetFile("../../data/multiome_train.parquet")
batch_group_size = multiome_file.metadata.num_rows // batch_num
for batch in tqdm(multiome_file.iter_batches(batch_size=batch_group_size)):
    multiome_batch = batch.to_pandas()
    multiome_batch = multiome_batch[~multiome_batch["location"].isin(gene_names)]

    multiome_batch["chromosome"] = multiome_batch["location"].apply(get_chr)
    multiome_batch["start"] = multiome_batch["location"].apply(get_start)
    multiome_batch["end"] = multiome_batch["location"].apply(get_end)

    multiome_df = pd.concat([multiome_df, multiome_batch], axis=0, ignore_index=True)

11it [25:02, 136.58s/it]


In [2]:
multiome_df.to_parquet("../../data/ATAC_train.parquet")

NameError: name 'multiome_df' is not defined

In [3]:
multiome_df = pd.read_parquet("../../data/temp/ATAC_train.parquet")

In [3]:
multiome_df

,obs_id,location,count,normalized_count,chromosome,start,end
0,000225c1151ab841,chr10:100346599-100347467,2,2.664076,chr10,100346599,100347467
1,000225c1151ab841,chr10:101214564-101215355,2,4.966185,chr10,101214564,101215355
2,000225c1151ab841,chr10:101353529-101354425,4,1.627825,chr10,101353529,101354425
3,000225c1151ab841,chr10:101830364-101831303,2,1.333610,chr10,101830364,101831303
4,000225c1151ab841,chr10:102064862-102065750,2,1.588739,chr10,102064862,102065750
...,...,...,...,...,...,...,...
173485197,ffffeba9dbf2bab1,chrY:19499847-19500746,1,3.342186,chrY,19499847,19500746
173485198,ffffeba9dbf2bab1,chrY:19566990-19567754,1,1.154666,chrY,19566990,19567754
173485199,ffffeba9dbf2bab1,chrY:20735890-20736626,3,7.049924,chrY,20735890,20736626
173485200,ffffeba9dbf2bab1,chrY:21260054-21260842,2,3.967924,chrY,21260054,21260842


In [4]:
multiome_obs_meta = pd.read_csv("../../data/multiome_obs_meta.csv")
B_cells_obs_ids = multiome_obs_meta[multiome_obs_meta["cell_type"] == "B cells"]["obs_id"].to_list()
T_cells_CD4_obs_ids = multiome_obs_meta[multiome_obs_meta["cell_type"] == "T cells CD4+"]["obs_id"].to_list()
T_cells_CD8_obs_ids = multiome_obs_meta[multiome_obs_meta["cell_type"] == "T cells CD8+"]["obs_id"].to_list()
Myeloid_obs_ids = multiome_obs_meta[multiome_obs_meta["cell_type"] == "Myeloid cells"]["obs_id"].to_list()
NK_obs_ids = multiome_obs_meta[multiome_obs_meta["cell_type"] == "NK cells"]["obs_id"].to_list()
T_regulatory_obs_ids = multiome_obs_meta[multiome_obs_meta["cell_type"] == "T regulatory cells"]["obs_id"].to_list()

In [5]:
B_cells_atac = multiome_df[multiome_df["obs_id"].isin(B_cells_obs_ids)]
T_cells_CD4_atac = multiome_df[multiome_df["obs_id"].isin(T_cells_CD4_obs_ids)]
T_cells_CD8_atac = multiome_df[multiome_df["obs_id"].isin(T_cells_CD8_obs_ids)]
Myeloid_atac = multiome_df[multiome_df["obs_id"].isin(Myeloid_obs_ids)]
NK_atac = multiome_df[multiome_df["obs_id"].isin(NK_obs_ids)]
T_regulatory_atac = multiome_df[multiome_df["obs_id"].isin(T_regulatory_obs_ids)] 

In [6]:
correlations = pd.read_table("../../data/allGeneCorrelations100000.p05_v3.txt")

In [7]:
sig_correlations = correlations[correlations["pval"]<=0.05]
sig_correlations

,dhs_chr,dhs_start,dhs_end,dhs_id,gene_chr,gene_start,gene_end,gene_name,metaprobeset_id,ensemblID,cor,pval
0,chrX,99861860,99862010,2861605,chrX,99885757,99891766,TSPAN6,1,ENSG00000000003,-0.544568,0.006
2,chrX,99880960,99881110,2861619,chrX,99885757,99891766,TSPAN6,1,ENSG00000000003,-0.402630,0.020
17,chrX,99942220,99942370,2861690,chrX,99885757,99891766,TSPAN6,1,ENSG00000000003,-0.385988,0.023
18,chrX,99942700,99942850,2861691,chrX,99885757,99891766,TSPAN6,1,ENSG00000000003,-0.446240,0.016
24,chrX,99870220,99870370,2861610,chrX,99839836,99854854,TNMD,2,ENSG00000000005,-0.065758,0.017
...,...,...,...,...,...,...,...,...,...,...,...,...
793518,chr15,83781325,83781475,931308,chr15,83691029,83691067,RP11-90B9.2,52291,ENSG00000259767,-0.169041,0.024
793519,chr15,83781480,83781630,931309,chr15,83691029,83691067,RP11-90B9.2,52291,ENSG00000259767,-0.245031,0.004
793526,chr16,72821420,72821570,1025241,chr16,72779322,72781064,RP5-991G20.1,52292,ENSG00000259768,-0.253807,0.021
793547,chr15,31528800,31528950,868131,chr15,31514639,31522962,RP11-16E12.2,52296,ENSG00000259772,-0.320208,0.005


In [8]:
sig_correlations.iloc[0,0]

'chrX'

In [7]:
ensemble_ids = list(set(list(sig_correlations["ensemblID"])))

In [10]:
sig_correlations

,dhs_chr,dhs_start,dhs_end,dhs_id,gene_chr,gene_start,gene_end,gene_name,metaprobeset_id,ensemblID,cor,pval
634736,chr7,10080,10230,2398536,chr7,71699,71819,AC093627.7,33334,ENSG00000232325,-0.343532,0.003
481648,chr17,14460,14610,1051511,chr17,31380,31404,DOC2B,16908,ENSG00000187939,-0.244659,0.010
683753,chr1,16140,16290,3,chr1,63656,63697,OR4G11P,39505,ENSG00000240361,-0.223318,0.011
695472,chr7,37380,37530,2398545,chr7,135911,149427,AC093627.9,41017,ENSG00000242474,-0.254091,0.011
634737,chr7,42445,42595,2398553,chr7,71699,71819,AC093627.7,33334,ENSG00000232325,-0.307110,0.014
...,...,...,...,...,...,...,...,...,...,...,...,...
516698,chr1,249201360,249201510,262476,chr1,249206914,249206945,U6,19119,ENSG00000200495,-0.068287,0.017
416425,chr1,249202585,249202735,262479,chr1,249106105,249111073,SH3BP5L,14012,ENSG00000175137,-0.185339,0.014
392632,chr1,249202785,249202935,262480,chr1,249144235,249153279,ZNF692,13161,ENSG00000171163,-0.258991,0.019
392633,chr1,249212885,249213035,262509,chr1,249144235,249153279,ZNF692,13161,ENSG00000171163,-0.258949,0.019


In [8]:
sig_correlations.sort_values(by="dhs_start", inplace=True)

C:\Users\ghanb\AppData\Local\Temp\ipykernel_25788\2579570082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_correlations.sort_values(by="dhs_start", inplace=True)


In [12]:
sig_correlations["dhs_chr"].unique()[0]

'chr7'

In [9]:
B_cells_atac.sort_values(by="start", inplace=True)

C:\Users\ghanb\AppData\Local\Temp\ipykernel_25788\159472389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B_cells_atac.sort_values(by="start", inplace=True)


In [61]:
B_cells_atac.groupby(by=["start", "end"])["normalized_count"].mean()

start      end      
2090       2982         2.288582
3945       4846         3.229936
4505       5401         3.923541
7910       8736         3.159859
9685       10586        3.067677
                          ...   
248866924  248867879    2.411894
248873533  248874443    0.886628
248906099  248906933    0.577764
248924630  248925480    1.573709
248945354  248946091    2.421512
Name: normalized_count, Length: 135154, dtype: float32

In [18]:
def create_gene_atac(atac, cor, ensemble_ids):
    row = 0
    gene_atac = pd.DataFrame(columns=["obs_id", "location", "chromosome", "ensembleID", "intersection", "cor_start", "cor_end", "score"])
    for id in tqdm(ensemble_ids):
        gene_ranges = cor[cor["ensemblID"]==id]
        chromosome = str(list(gene_ranges["dhs_chr"])[0])
        atac = atac[atac["chromosome"]==chromosome]
        for i in range(len(gene_ranges)):
            seq_range = range(int(gene_ranges["dhs_start"].iloc[i]), int(gene_ranges["dhs_end"].iloc[i])+1)

            for j in range(len(atac)):
                location_range = range(int(atac["start"].iloc[j]), int(atac["end"].iloc[j])+1)
                ranges_intersection = len(set(seq_range).intersection(location_range))
                if ranges_intersection>0:
                    gene_atac.iloc[row, 0] = atac["obs_id"].iloc[j]
                    gene_atac.iloc[row, 1] = atac["location"].iloc[j]
                    gene_atac.iloc[row, 2] = atac["chromosome"].iloc[j]
                    gene_atac.iloc[row, 3] = id
                    gene_atac.iloc[row, 4] = ranges_intersection/len(seq_range)
                    gene_atac.iloc[row, 5] = gene_ranges["dhs_start"].iloc[i]
                    gene_atac.iloc[row, 6] = gene_ranges["dhs_end"].iloc[i]
                    gene_atac.iloc[row, 7] = atac["normalized_count"].iloc[j]

                    row+=1
                    
    return gene_atac        
                    
                
            
            

    
    

In [25]:
def create_gene_atac2(atac, cor):
    gene_atac = pd.DataFrame(np.zeros((len(cor), 7)))
    gene_atac.columns = ["obs_id", "chromosome", "ensembleID", "intersection", "cor_start", "cor_end", "score"]
    all_cor = cor.copy()
    all_atac = atac.copy()
    gene_atac_row = 0

    for chromosome in tqdm(all_cor["dhs_chr"].unique()):
        atac = all_atac[all_atac["chromosome"]==chromosome]
        cor = all_cor[all_cor["dhs_chr"]==chromosome]

        pointer = 0
        direction = -1
        
        for i, atac_row in atac.iterrows():
            direction*=-1
            atac_start = atac_row["start"]
            atac_end = atac_row["end"]   
            flag = 0 
              
            while pointer<len(cor) and flag !=2 and gene_atac_row<=len(all_cor):
                #if gene_atac_row==len(all_cor)+1:
                    #return gene_atac
                if atac_start < cor["dhs_end"].iloc[pointer] and atac_end > cor["dhs_start"].iloc[pointer]:
                    flag = 1
                    seq_range = range(int(cor["dhs_start"].iloc[pointer]), int(cor["dhs_end"].iloc[pointer])+1)
                    location_range = range(atac_start, atac_end+1)
                    ranges_intersection = len(set(seq_range).intersection(location_range))

                    gene_atac.iloc[gene_atac_row, 0] = atac_row["obs_id"]
                    gene_atac.iloc[gene_atac_row, 1] = chromosome
                    gene_atac.iloc[gene_atac_row, 2] = cor["ensemblID"].iloc[pointer]
                    gene_atac.iloc[gene_atac_row, 3] = ranges_intersection/len(seq_range)
                    gene_atac.iloc[gene_atac_row, 4] = cor["dhs_start"].iloc[pointer]
                    gene_atac.iloc[gene_atac_row, 5] = cor["dhs_end"].iloc[pointer]
                    gene_atac.iloc[gene_atac_row, 6] = atac_row["normalized_count"]

                    pointer+=1
                    gene_atac_row+=1
                    print(pointer)
                else:
                    if flag == 0:
                        
                        if direction==-1:
                            pointer+=1
                        else:
                            pointer-=1
                        
                    else:
                        flag = 2
    return gene_atac
    
    
    


In [54]:
def create_gene_atac3(atac, cor):
    gene_atac = pd.DataFrame(columns=["obs_id", "chromosome", "ensembleID", "intersection", "score"])
    # gene_atac.columns = ["obs_id", "chromosome", "ensembleID", "intersection", "cor_start", "cor_end", "score"]
    all_cor = cor.copy()
    all_atac = atac.copy()

    for cn, chromosome in enumerate(all_cor["dhs_chr"].unique()[9:11]):
        atac = all_atac[all_atac["chromosome"]==chromosome]
        cor = all_cor[all_cor["dhs_chr"]==chromosome]
        print(f"chromosome no: {cn} | name: {chromosome}")
        print(f"atac len for chromosome: {len(atac)}")
        print(f"cor len for chromosome: {len(cor)}")

        p_atac, p_cor = 0, 0

        ### ALIGN
        ###

        if cor["dhs_start"].iloc[p_cor] > atac["end"].iloc[p_atac]:
            # while not (atac["start"].iloc[p_atac] < cor["dhs_end"].iloc[p_cor] and \
            #            atac["end"].iloc[p_atac] > cor["dhs_start"].iloc[p_cor]):
            while cor["dhs_start"].iloc[p_cor] > atac["end"].iloc[p_atac]:
                p_atac += 1
        elif cor["dhs_end"].iloc[p_cor] < atac["start"].iloc[p_atac]:
            # while not (atac["start"].iloc[p_atac] < cor["dhs_end"].iloc[p_cor] and \
            #            atac["end"].iloc[p_atac] > cor["dhs_start"].iloc[p_cor]):
            while cor["dhs_end"].iloc[p_cor] < atac["start"].iloc[p_atac]:
                p_cor += 1

        while p_cor < len(cor) and p_atac < len(atac): # shart e kharej shodan
            
            
            if p_cor%1000==0:
                print(p_cor, end=", ")
            while atac["start"].iloc[p_atac] < cor["dhs_end"].iloc[p_cor] and \
                atac["end"].iloc[p_atac] > cor["dhs_start"].iloc[p_cor]:

                
                obs_id = atac["obs_id"].iloc[p_atac]
                # chromosome
                ensemble_id = cor["ensemblID"].iloc[p_cor]
                dhs_start = cor["dhs_start"].iloc[p_cor]
                dhs_end = cor["dhs_end"].iloc[p_cor]
                atac_start = atac["start"].iloc[p_atac]
                atac_end = atac["end"].iloc[p_atac]
                intersection = (min(atac_end, dhs_end)-(max(atac_start, dhs_start)))/(dhs_end-dhs_start)

                normalized_count = atac["normalized_count"].iloc[p_atac]
                series = pd.DataFrame(
                    {
                        "obs_id": [obs_id],
                        "chromosome": [chromosome],
                        "ensembleID": [ensemble_id],
                        "intersection": [intersection],
                        "score": [normalized_count]
                    }
                )
                gene_atac = pd.concat([gene_atac, series], axis=0)
                

                p_cor += 1
                if p_cor==len(cor):
                    p_cor-=1
                    break
            p_atac += 1
            if p_atac==len(atac):
                p_atac-=1
                break
            while atac["start"].iloc[p_atac]==atac["start"].iloc[p_atac-1] and \
                atac["end"].iloc[p_atac]==atac["end"].iloc[p_atac-1]:
                p_atac+=1
                if p_atac==len(atac):
                    p_atac-=1
                    break

            if p_atac == len(atac):
                break

            if cor["dhs_start"].iloc[p_cor] > atac["end"].iloc[p_atac]:
                while cor["dhs_start"].iloc[p_cor] > atac["end"].iloc[p_atac]:
                    # print("hiii", p_cor)
                    p_cor -= 1
                while atac["start"].iloc[p_atac] < cor["dhs_end"].iloc[p_cor] and \
                    atac["end"].iloc[p_atac] > cor["dhs_start"].iloc[p_cor]:
                    p_cor -= 1
                p_cor += 1
                if p_cor==len(cor):
                    p_cor-=1
                    break
                continue
            if cor["dhs_end"].iloc[p_cor] < atac["start"].iloc[p_atac]:
                while cor["dhs_end"].iloc[p_cor] < atac["start"].iloc[p_atac]:
                    p_cor += 1
                    if p_cor == len(cor):
                        p_cor -= 1
                        break
                    
                continue
            while atac["start"].iloc[p_atac] < cor["dhs_end"].iloc[p_cor] and \
                atac["end"].iloc[p_atac] > cor["dhs_start"].iloc[p_cor]:
                p_cor -= 1
            p_cor += 1
            if p_cor == len(cor):
                p_cor -= 1
                break
            continue
    
    return gene_atac

In [55]:
B_cells_gene_atac = create_gene_atac3(B_cells_atac, sig_correlations)

chromosome no: 0 | name: chr10
atac len for chromosome: 1224389
cor len for chromosome: 6753
0, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, chromosome no: 1 | name: chr6
atac len for chromosome: 1704688
cor len for chromosome: 9489
0, 4000, 8000, 8000, 8000, 

In [56]:
B_cells_gene_atac

,obs_id,chromosome,ensembleID,intersection,score
0,05fadfb8107d4e0c,chr10,ENSG00000151240,1.000000,2.097998
0,7af4f888f5661a62,chr10,ENSG00000239822,1.000000,1.443676
0,1a69bd9e461e00b7,chr10,ENSG00000047056,1.000000,0.932475
0,52e77bb794556848,chr10,ENSG00000237399,1.000000,0.760328
0,21462ae0a553a6ae,chr10,ENSG00000236892,0.820000,3.683643
...,...,...,...,...,...
0,2bb210b00604ce9e,chr6,ENSG00000184786,1.000000,0.880386
0,2bb210b00604ce9e,chr6,ENSG00000130024,1.000000,0.880386
0,c0c2d3413321e293,chr6,ENSG00000185127,1.000000,1.647786
0,2b8b1ac60e0f82e0,chr6,ENSG00000236173,0.820000,1.507023


In [18]:
B_cells_atac[B_cells_atac["chromosome"]=="chr17"]

,obs_id,location,count,normalized_count,chromosome,start,end
134540387,c50c5ab93f69ce46,chr17:112841-113741,2,1.819549,chr17,112841,113741
138524130,cb8174035d4e5865,chr17:112841-113741,2,1.480782,chr17,112841,113741
127592517,ba7b8dc1116cf085,chr17:112841-113741,2,1.734908,chr17,112841,113741
151103920,de976dabfb43f494,chr17:112841-113741,2,1.121243,chr17,112841,113741
9292552,0d6386d9fe2cd66d,chr17:112841-113741,2,1.573857,chr17,112841,113741
...,...,...,...,...,...,...,...
153582610,e23ab6cb65282945,chr17:83246798-83247700,2,3.337099,chr17,83246798,83247700
132216518,c18ef8ec5c239e07,chr17:83246798-83247700,2,1.937325,chr17,83246798,83247700
5941064,08b42f34074d30cd,chr17:83246798-83247700,2,2.060075,chr17,83246798,83247700
38964356,387dde2de6c538b8,chr17:83246798-83247700,2,2.254571,chr17,83246798,83247700


In [19]:
sig_correlations[sig_correlations["dhs_chr"]=="chr17"]

,dhs_chr,dhs_start,dhs_end,dhs_id,gene_chr,gene_start,gene_end,gene_name,metaprobeset_id,ensemblID,cor,pval
481648,chr17,14460,14610,1051511,chr17,31380,31404,DOC2B,16908,ENSG00000187939,-0.244659,0.010
481651,chr17,49180,49330,1051584,chr17,31380,31404,DOC2B,16908,ENSG00000187939,-0.242436,0.010
481652,chr17,49800,49950,1051585,chr17,31380,31404,DOC2B,16908,ENSG00000187939,-0.308090,0.003
481653,chr17,49980,50130,1051586,chr17,31380,31404,DOC2B,16908,ENSG00000187939,-0.260791,0.006
481654,chr17,50160,50310,1051587,chr17,31380,31404,DOC2B,16908,ENSG00000187939,-0.287450,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...
424856,chr17,81128545,81128695,1167186,chr17,81037578,81043172,METRNL,14362,ENSG00000176845,-0.329763,0.022
638640,chr17,81137460,81137610,1167210,chr17,81174783,81174903,AC139099.1,33812,ENSG00000232938,-0.073778,0.018
638641,chr17,81137640,81137790,1167211,chr17,81174783,81174903,AC139099.1,33812,ENSG00000232938,-0.063751,0.023
638642,chr17,81150865,81151015,1167255,chr17,81174783,81174903,AC139099.1,33812,ENSG00000232938,-0.074219,0.018


In [20]:
def create_gene_atac4(atac, cor):
    gene_atac = pd.DataFrame()
    all_cor = cor.copy()
    all_atac = atac.copy()
    
    for cn, chromosome in enumerate(all_cor["dhs_chr"].unique()):
        atac = all_atac[all_atac["chromosome"]==chromosome]
        cor = all_cor[all_cor["dhs_chr"]==chromosome]
        print(f"chromosome no: {cn} | name: {chromosome}")
        print(f"atac len for chromosome: {len(atac)}")
        print(f"cor len for chromosome: {len(cor)}")

        p_atac, p_cor = 0, 0

        

ValueError: Grouper for '<class 'pandas.core.frame.DataFrame'>' not 1-dimensional

In [9]:
B_cells_sig_start = np.array(sig_correlations["gene_start"]).reshape(1, -1)
B_cells_sig_end = np.array(sig_correlations["gene_end"]).reshape(1, -1)
B_cells_atac_start = np.array(B_cells_atac["start"]).reshape(-1, 1)
B_cells_atac_end = np.array(B_cells_atac["end"]).reshape(-1, 1)
B_cells_atac_Ncount = np.array(B_cells_atac["normalized_count"]).reshape(-1, 1)
sig_genes = sig_correlations["ensemblID"].to_list()
# B_cells_broadcast = np.broadcast(sig_array, B_cells_atac_array[:len(sig_array)])
# out = np.empty(B_cells_broadcast.shape)
#out.flat = [min(int(sig_end), int(atac_end))-max(int(sig_start), int(atac_start)+1) for ((sig_start, sig_end), (atac_start, atac_end)) in B_cells_broadcast]
#out

In [11]:
def compute_sig_access(sig_start, sig_end, atac_starts, atac_ends, atac_Ncounts):
    overlap = (np.minimum(sig_end, atac_ends) - np.maximum(sig_start, atac_starts))/(sig_end-sig_start)
    overlap[overlap<0]=0
    s = overlap.sum()
    if s > 0:
        accessScore = (overlap * atac_Ncounts).sum() / overlap.sum()
    else:
        accessScore = -1
         
    return accessScore

In [10]:
import temp
from temp import test

In [14]:
from multiprocessing import Pool
from temp import test

if __name__ == "__main__":

    def get_gene_accessibility(sig_starts, sig_ends, atac_starts, atac_ends, atac_Ncounts, sig_genes): 
        gene_accessibility = pd.DataFrame(columns=["ensembleID", "accessibility"])
        
        batch_size = 1000
        batch_num = (sig_starts.shape[1] // batch_size) + 1
        for i in tqdm(range(batch_num)):
            if i < batch_num - 1:
                sig_start = sig_starts[0, i*batch_size:(i+1)*batch_size]
                sig_end = sig_ends[0, i*batch_size:(i+1)*batch_size]
            else:
                sig_start = sig_starts[0, i*batch_size:]
                sig_end = sig_ends[0, i*batch_size:]
            
            c = test(atac_starts, atac_ends, atac_Ncounts)
            p = Pool(16)
            access = p.starmap(c.compute_sig_access, zip(list(sig_start), list(sig_end)))
            gene = sig_genes[i]
            gene_accessibility = pd.concat([gene_accessibility, pd.DataFrame({"gene":gene, "accessibility":access})])
        return gene_accessibility
    
    B_cells_gene_accessibility = get_gene_accessibility(B_cells_sig_start, B_cells_sig_end, B_cells_atac_start, B_cells_atac_end, B_cells_atac_Ncount, sig_genes)

  0%|          | 0/177 [00:00<?, ?it/s]

In [1]:
from multiprocessing import Pool

In [3]:
from temp import f

if __name__ == "__main__":
    p = Pool(3)
    params = [(1, 2), (2, 3), (3, 4)]
    results = p.starmap(f, params)

In [4]:
results

[2, 6, 12]

results[0].get()

In [44]:
d.keys()

[]

In [14]:
B_cells_gene_accessibility = get_gene_accessibility(B_cells_sig_start, B_cells_sig_end, B_cells_atac_start, B_cells_atac_end, B_cells_atac_Ncount, sig_genes)

  0%|          | 0/8818 [00:00<?, ?it/s]C:\Users\yayag\AppData\Local\Temp\ipykernel_19228\2134768079.py:4: RuntimeWarning: invalid value encountered in divide
  accessScore = (overlap * atac_Ncounts).sum(0) / overlap.sum(0)
C:\Users\yayag\AppData\Local\Temp\ipykernel_19228\470227413.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gene_accessibility = pd.concat([gene_accessibility, pd.DataFrame({"gene":gene, "accessibility":access})])
  0%|          | 1/8818 [00:49<121:36:53, 49.66s/it]C:\Users\yayag\AppData\Local\Temp\ipykernel_19228\2134768079.py:4: RuntimeWarning: invalid value encountered in divide
  accessScore = (overlap * atac_Ncounts).sum(0) / overlap.sum(0)
  0%|          | 2/8818 [01:08<77:05:10, 31.48s/it] C:\Users\yayag\AppD

KeyboardInterrupt: 

In [61]:
sig_genes[0]

'ENSG00000232325'

In [45]:
sth[sth<0]=0
(sth * B_cells_atac_Ncount).sum() / sth.sum()

2.4062807866574807

In [15]:
x, y = sig_array[0]

In [16]:
x

71699

In [27]:
B_cells_atac[B_cells_atac["chromosome"]=="chr2"]

,obs_id,location,count,normalized_count,chromosome,start,end
162859353,f03f7ded47ff1b35,chr2:9976-10826,2,3.021235,chr2,9976,10826
126050035,b838a74cdfdb1634,chr2:9976-10826,2,2.050489,chr2,9976,10826
116961257,aa74fcd6c46bd74e,chr2:9976-10826,2,1.265548,chr2,9976,10826
163146930,f0baf396e3ab61a5,chr2:9976-10826,2,5.448502,chr2,9976,10826
146133662,d7352ad054c67b77,chr2:9976-10826,2,6.470487,chr2,9976,10826
...,...,...,...,...,...,...,...
49652552,47eb9c3e01c6a601,chr2:242182974-242183864,2,4.400481,chr2,242182974,242183864
13266114,12c2025cf0abd959,chr2:242182974-242183864,2,2.790936,chr2,242182974,242183864
23090053,2181f8df45af4076,chr2:242182974-242183864,2,1.426278,chr2,242182974,242183864
105988782,9af75c60c38378f5,chr2:242182974-242183864,2,1.777322,chr2,242182974,242183864


In [30]:
B_cells_gene_atac[B_cells_gene_atac["chromosome"]!=0]

,obs_id,chromosome,ensembleID,intersection,cor_start,cor_end,score
0,841a3e50861e4ab1,chr7,ENSG00000232325,1.0,10080.0,10230.0,3.898647
2,4240bac01e20b43e,chr7,ENSG00000232325,1.0,10080.0,10230.0,1.994656
4,444903c060e9f7af,chr7,ENSG00000232325,1.0,10080.0,10230.0,4.962905
6,fec6b653a30382fc,chr7,ENSG00000232325,1.0,10080.0,10230.0,4.186073
8,7e37aae9075ffeda,chr7,ENSG00000232325,1.0,10080.0,10230.0,2.736343
10,b88c4f893601c319,chr7,ENSG00000232325,1.0,10080.0,10230.0,1.371382
12,ea503ec86c1aa28a,chr7,ENSG00000232325,1.0,10080.0,10230.0,2.176123
14,e35b3f5df751962b,chr7,ENSG00000232325,1.0,10080.0,10230.0,2.187354
16,98c7a5622654d820,chr7,ENSG00000232325,1.0,10080.0,10230.0,3.953332


In [21]:
sig_correlations["dhs_end"].iloc[0]

10230

In [20]:
B_cells_gene_atac

,obs_id,chromosome,ensembleID,intersection,cor_start,cor_end,score
0,841a3e50861e4ab1,chr7,ENSG00000232325,1.0,10080.0,10230.0,3.898647
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,3dc613637024e42b,chr7,ENSG00000232325,1.0,10080.0,10230.0,2.142502
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...
176347,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
176348,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
176349,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
176350,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [39]:
len(set(range(1,11)).intersection(set(range(10,15))))

1

In [29]:
gene_names.index("RP11")

ValueError: 'RP11' is not in list

In [3]:
atac = multiome_df.loc[258406:]
atac

,obs_id,location,count,normalized_count
258406,004dd1a01106608a,chr10:100006075-100006963,2,2.479890
258407,004dd1a01106608a,chr10:100229082-100229931,1,0.824665
258408,004dd1a01106608a,chr10:100286187-100287039,2,0.984346
258409,004dd1a01106608a,chr10:100339340-100339978,2,3.327292
258410,004dd1a01106608a,chr10:101066169-101066970,4,3.590057
...,...,...,...,...
216251363,ffffeba9dbf2bab1,chrY:19499847-19500746,1,3.342186
216251364,ffffeba9dbf2bab1,chrY:19566990-19567754,1,1.154666
216251365,ffffeba9dbf2bab1,chrY:20735890-20736626,3,7.049924
216251366,ffffeba9dbf2bab1,chrY:21260054-21260842,2,3.967924


In [4]:
def get_chr(all_str):
    index = all_str.find(":") 
    return all_str[:index] 


atac["chromosome"] = atac["location"].apply(get_chr)

C:\Users\yayag\AppData\Local\Temp\ipykernel_9316\820159844.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atac["chromosome"] = atac["location"].apply(get_chr)


In [5]:
def get_start(all_str):
    return all_str[all_str.find(":")+1:all_str.find("-")] 


atac["start"] = atac["location"].apply(get_start)

C:\Users\yayag\AppData\Local\Temp\ipykernel_9316\2006402745.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atac["start"] = atac["location"].apply(get_start)


KeyboardInterrupt: 

In [8]:
atac.drop("count", axis=1, inplace=True)

C:\Users\yayag\AppData\Local\Temp\ipykernel_9316\905927124.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atac.drop("count", axis=1, inplace=True)


In [9]:
atac

,obs_id,location,normalized_count,chromosome,start
258406,004dd1a01106608a,chr10:100006075-100006963,2.479890,chr10,100006075
258407,004dd1a01106608a,chr10:100229082-100229931,0.824665,chr10,100229082
258408,004dd1a01106608a,chr10:100286187-100287039,0.984346,chr10,100286187
258409,004dd1a01106608a,chr10:100339340-100339978,3.327292,chr10,100339340
258410,004dd1a01106608a,chr10:101066169-101066970,3.590057,chr10,101066169
...,...,...,...,...,...
216251363,ffffeba9dbf2bab1,chrY:19499847-19500746,3.342186,chrY,19499847
216251364,ffffeba9dbf2bab1,chrY:19566990-19567754,1.154666,chrY,19566990
216251365,ffffeba9dbf2bab1,chrY:20735890-20736626,7.049924,chrY,20735890
216251366,ffffeba9dbf2bab1,chrY:21260054-21260842,3.967924,chrY,21260054


In [10]:
def get_end(all_str):
    return all_str[all_str.find("-")+1:] 


atac["end"] = atac["location"].apply(get_end)